In [1]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

%matplotlib inline

Using TensorFlow backend.


In [17]:
df = pd.read_csv("fer2013.csv")

In [18]:
df["Usage"].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [4]:
train = df[["emotion", "pixels"]][df["Usage"] == "Training"]
train.isnull().sum()

emotion    0
pixels     0
dtype: int64

In [5]:
train['pixels'] = train['pixels'].apply(lambda im: np.fromstring(im, sep=' '))
x_train = np.vstack(train['pixels'].values)
y_train = np.array(train["emotion"])
x_train.shape, y_train.shape

((28709, 2304), (28709,))

In [6]:
public_test_df = df[["emotion", "pixels"]][df["Usage"]=="PublicTest"]

In [7]:
public_test_df["pixels"] = public_test_df["pixels"].apply(lambda im: np.fromstring(im, sep=' '))
x_test = np.vstack(public_test_df["pixels"].values)
y_test = np.array(public_test_df["emotion"])

In [8]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)
x_train.shape, x_test.shape

((28709, 48, 48, 1), (3589, 48, 48, 1))

In [9]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_train.shape, y_test.shape

((28709, 7), (3589, 7))

In [11]:
model = Sequential()
model.add(Conv2D(64, 3, data_format="channels_last", kernel_initializer="he_normal", 
                 input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(64, 3))

model.add(BatchNormalization())
model.add(Activation("relu"))


model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.6))
model.add(Conv2D(32, 3))

model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(32, 3))

model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(32, 3))
model.add(BatchNormalization())

model.add(Activation("relu"))

model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.6))

model.add(Flatten())

model.add(Dense(128))

model.add(BatchNormalization())

model.add(Activation("relu"))
model.add(Dropout(0.6))
model.add(Dense(7))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_v2 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 44, 44, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0

In [12]:
# save best weights
checkpointer = ModelCheckpoint(filepath='face_model.h5', verbose=1, save_best_only=True)

# num epochs
epochs = 10

# run model
hist = model.fit(x_train, y_train, epochs=epochs,
                 shuffle=True,
                 batch_size=100, validation_data=(x_test, y_test),
                 callbacks=[checkpointer], verbose=2)

# save model to json
model_json = model.to_json()
with open("face_model.json", "w") as json_file:
    json_file.write(model_json)

Train on 28709 samples, validate on 3589 samples
Epoch 1/10

Epoch 00001: val_loss improved from inf to 1.63628, saving model to face_model.h5
28709/28709 - 767s - loss: 2.0127 - accuracy: 0.2511 - val_loss: 1.6363 - val_accuracy: 0.3700
Epoch 2/10

Epoch 00002: val_loss improved from 1.63628 to 1.50763, saving model to face_model.h5
28709/28709 - 622s - loss: 1.6539 - accuracy: 0.3529 - val_loss: 1.5076 - val_accuracy: 0.4199
Epoch 3/10

Epoch 00003: val_loss improved from 1.50763 to 1.49847, saving model to face_model.h5
28709/28709 - 622s - loss: 1.5344 - accuracy: 0.4006 - val_loss: 1.4985 - val_accuracy: 0.4310
Epoch 4/10

Epoch 00004: val_loss improved from 1.49847 to 1.40611, saving model to face_model.h5
28709/28709 - 628s - loss: 1.4616 - accuracy: 0.4374 - val_loss: 1.4061 - val_accuracy: 0.4533
Epoch 5/10

Epoch 00005: val_loss improved from 1.40611 to 1.33396, saving model to face_model.h5
28709/28709 - 622s - loss: 1.4099 - accuracy: 0.4575 - val_loss: 1.3340 - val_accurac

In [19]:
import cv2
from model import FacialExpressionModel
import numpy as np

rgb = cv2.VideoCapture(0)
facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX

def __get_data__():
    """
    __get_data__: Gets data from the VideoCapture object and classifies them
    to a face or no face. 
    
    returns: tuple (faces in image, frame read, grayscale frame)
    """
    _, fr = rgb.read()
    gray = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    faces = facec.detectMultiScale(gray, 1.3, 5)
    
    return faces, fr, gray

def start_app(cnn):
    skip_frame = 10
    data = []
    flag = False
    ix = 0
    while True:
        ix += 1
        
        faces, fr, gray_fr = __get_data__()
        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]
            
            roi = cv2.resize(fc, (48, 48))
            pred = cnn.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        if cv2.waitKey(1) == 27:
            break
        cv2.imshow('Filter', fr)
    cv2.destroyAllWindows()



In [20]:
model = FacialExpressionModel("face_model.json", "face_model.h5")
start_app(model)

Model loaded from disk
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_v2 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 44, 44, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (Non